In [1]:
library(RPostgreSQL)
library(tidyverse)

Loading required package: DBI

To reproduce results from prior versions of the twang package, please see the version="legacy" option described in the documentation.

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.1     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.0     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
data_dir <- file.path("..", "data")
sql_dir <- file.path("..", "sql")

In [3]:
drv <- dbDriver("PostgreSQL")
con <- dbConnect(drv, dbname = "mimic")
dbSendQuery(con, "set search_path=echo,public,mimiciii;")

<PostgreSQLResult>

In [4]:
full_data <- dbGetQuery(con, "select * from merged_data")
head(full_data)

,icustay_id,hadm_id,subject_id,first_careunit,intime,outtime,angus,age,icu_order,echo_time,⋯,lab_lactate_flag,lab_hemoglobin_flag,lab_sodium_flag,lab_creatinine_kinase_flag,lab_ph_flag,lab_bnp_flag,lab_bicarbonate_flag,lab_wbc_flag,lab_bun_flag,lab_pco2_flag
,<int>,<int>,<int>,<chr>,<dttm>,<dttm>,<int>,<dbl>,<dbl>,<dttm>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,238250,106084,28940,MICU,2107-05-01 00:06:13,2107-05-13,1,61.25803,1,NA,⋯,0,1,1,0,0,0,1,1,1,0
2,273462,124592,24082,SICU,2178-04-09 17:37:06,2178-04-12,1,56.18668,1,NA,⋯,1,1,1,0,1,0,1,1,1,1
3,282609,108445,16536,MICU,2143-04-11 00:51:57,2143-04-14,1,52.86642,1,2143-04-11,⋯,1,1,1,1,1,0,1,1,1,1
4,297366,115424,8511,MICU,2130-12-18 14:31:59,2130-12-19,1,69.99635,1,NA,⋯,0,1,1,0,0,0,1,1,1,0
5,232920,107703,92623,MICU,2163-03-31 05:07:26,2163-04-01,1,91.50000,1,NA,⋯,0,1,1,1,0,0,1,1,1,0
6,263138,119476,27996,SICU,2171-01-12 05:05:24,2171-01-13,1,51.44592,1,NA,⋯,1,1,1,0,0,0,1,1,1,0


In [5]:
dbGetQuery(con, "select count(*) from merged_data")

,count
,<dbl>
1,6361


In [4]:
full_data <- dbGetQuery(con, "select * from merged_data")
head(full_data)

icustay_id,hadm_id,subject_id,first_careunit,intime,outtime,angus,age,icu_order,echo_time,⋯,lab_sodium_flag,lab_bun_flag,lab_bicarbonate_flag,lab_bnp_flag,lab_pco2_flag,lab_creatinine_flag,lab_potassium_flag,lab_troponin_flag,lab_po2_flag,lab_lactate_flag
228416,159895,5491,MICU,2151-06-29 01:18:27,2151-07-27 16:00:56,1,76.58225,1,2151-06-30,⋯,1,1,1,0,1,1,1,0,1,1
278148,169700,49395,MICU,2197-01-09 00:45:42,2197-01-10 06:46:29,1,82.72606,1,NA,⋯,1,1,1,0,0,1,1,0,0,0
274174,103722,14855,MICU,2181-09-08 23:43:13,2181-09-26 16:52:02,1,61.84390,1,2181-09-11,⋯,1,1,1,0,1,1,1,0,1,1
214954,170515,54642,MICU,2176-03-11 04:58:23,2176-03-15 18:00:48,1,63.86507,1,2176-03-11,⋯,1,1,1,0,1,1,1,1,1,1
222457,183493,96815,MICU,2105-06-04 20:57:55,2105-06-06 20:01:51,1,62.84018,1,NA,⋯,1,1,1,0,0,1,1,0,0,0
264413,134244,81436,MICU,2133-03-27 16:31:37,2133-04-04 16:15:28,1,62.60148,1,2133-03-31,⋯,1,1,1,0,0,1,1,0,0,1


In [11]:
dim(full_data)

[1] 6361  132

In [14]:
write.csv(full_data, file="../data/preprocessed.csv")

In [13]:
class(full_data)

[1] "data.frame"

In [ ]:
feature_names %>%
data.frame(feature = ., stringsAsFactors = FALSE) %>%
data.table::fwrite(file = file.path(data_dir, "features.csv"))

In [5]:
dbDisconnect(con)
dbUnloadDriver(drv)

[1] TRUE

[1] TRUE

In [6]:
to_factor <- function(x) {
    if(length(na.omit(unique(x))) <= 1) return(factor(x, levels = c(0, 1)))
    return(factor(x))
}

In [7]:
factor_vars <- full_data %>%
    names %>%
    grep("flag|abnormal|icd|sedative", ., value = TRUE) %>%
    c("gender", "first_careunit", "echo", "vent", "vaso",
      "icu_adm_hour", "mort_28_day")
factor_vars

[1] "icd_chf"                        "icd_afib"                      
 [3] "icd_renal"                      "icd_liver"                     
 [5] "icd_copd"                       "icd_cad"                       
 [7] "icd_stroke"                     "icd_malignancy"                
 [9] "lab_chloride_abnormal"          "lab_creatinine_abnormal"       
[11] "lab_platelet_abnormal"          "lab_potassium_abnormal"        
[13] "lab_troponin_abnormal"          "lab_po2_abnormal"              
[15] "lab_lactate_abnormal"           "lab_hemoglobin_abnormal"       
[17] "lab_sodium_abnormal"            "lab_creatinine_kinase_abnormal"
[19] "lab_ph_abnormal"                "lab_bnp_abnormal"              
[21] "lab_bicarbonate_abnormal"       "lab_wbc_abnormal"              
[23] "lab_bun_abnormal"               "lab_pco2_abnormal"             
[25] "sedative"                       "vs_cvp_flag"                   
[27] "vs_heart_rate_flag"             "vs_map_flag"                   
[29] "vs_temp_flag"                   "lab_chloride_flag"             
[31] "lab_creatinine_flag"            "lab_platelet_flag"             
[33] "lab_potassium_flag"             "lab_troponin_flag"             
[35] "lab_po2_flag"                   "lab_lactate_flag"              
[37] "lab_hemoglobin_flag"            "lab_sodium_flag"               
[39] "lab_creatinine_kinase_flag"     "lab_ph_flag"                   
[41] "lab_bnp_flag"                   "lab_bicarbonate_flag"          
[43] "lab_wbc_flag"                   "lab_bun_flag"                  
[45] "lab_pco2_flag"                  "gender"                        
[47] "first_careunit"                 "echo"                          
[49] "vent"                           "vaso"                          
[51] "icu_adm_hour"                   "mort_28_day"

In [8]:
weekday <- c("sun", "mon", "tues", "wednes", "thurs", "fri", "satur") %>%
paste0("day")
weekday

[1] "sunday"    "monday"    "tuesday"   "wednesday" "thursday"  "friday"   
[7] "saturday"

In [9]:
full_data <- full_data %>%
    mutate(echo_int = as.integer(echo)) %>%
    mutate(icu_adm_weekday = weekday[icu_adm_weekday + 1]) %>%
    mutate(mort_28_day_int = as.integer(mort_28_day)) %>%
    mutate_at(factor_vars, to_factor) %>%
    mutate(icu_adm_weekday = factor(icu_adm_weekday, levels = weekday)) %>%
    mutate(gender = relevel(gender, "M")) %>%
    mutate(first_careunit = relevel(first_careunit, "SICU"))
full_data %>% pull(echo) %>% head

[1] 0 0 1 0 0 0
Levels: 0 1

In [10]:
feature_names <- full_data %>%
    names %>%
    keep(grepl("vs|lab|icd|age|gender|weight|saps|sofa|elix_score|vent|vaso|icu_adm|careunit|sedative", .)) %>%
    discard(grepl("vs|lab", .) & grepl("flag", .) & !grepl("bnp|troponin|kinase|cvp", .)) %>%
    discard(grepl("bnp|troponin|kinase|cvp", .) & !grepl("flag", .)) %>%
    discard(grepl("min|max", .)) %>%
    discard(grepl("abnormal", .))
feature_names
length(feature_names)

[1] "first_careunit"             "age"                       
 [3] "gender"                     "weight"                    
 [5] "saps"                       "sofa"                      
 [7] "elix_score"                 "vent"                      
 [9] "vaso"                       "icu_adm_weekday"           
[11] "icu_adm_hour"               "icd_chf"                   
[13] "icd_afib"                   "icd_renal"                 
[15] "icd_liver"                  "icd_copd"                  
[17] "icd_cad"                    "icd_stroke"                
[19] "icd_malignancy"             "vs_heart_rate_first"       
[21] "vs_map_first"               "vs_temp_first"             
[23] "lab_chloride_first"         "lab_creatinine_first"      
[25] "lab_platelet_first"         "lab_potassium_first"       
[27] "lab_po2_first"              "lab_lactate_first"         
[29] "lab_hemoglobin_first"       "lab_sodium_first"          
[31] "lab_ph_first"               "lab_bicarbonate_first"     
[33] "lab_wbc_first"              "lab_bun_first"             
[35] "lab_pco2_first"             "sedative"                  
[37] "vs_cvp_flag"                "lab_troponin_flag"         
[39] "lab_creatinine_kinase_flag" "lab_bnp_flag"

[1] 40

In [12]:
full_data %>% colnames

[1] "icustay_id"                     "hadm_id"                       
  [3] "subject_id"                     "first_careunit"                
  [5] "intime"                         "outtime"                       
  [7] "angus"                          "age"                           
  [9] "icu_order"                      "echo_time"                     
 [11] "echo_exclude"                   "echo_include"                  
 [13] "echo"                           "gender"                        
 [15] "weight"                         "saps"                          
 [17] "sofa"                           "elix_score"                    
 [19] "vent"                           "vaso"                          
 [21] "icu_adm_weekday"                "icu_adm_hour"                  
 [23] "deathtime"                      "mort_28_day"                   
 [25] "icu_los_day"                    "icd_chf"                       
 [27] "icd_afib"                       "icd_renal"                     
 [29] "icd_liver"                      "icd_copd"                      
 [31] "icd_cad"                        "icd_stroke"                    
 [33] "icd_malignancy"                 "vs_cvp_first"                  
 [35] "vs_cvp_min"                     "vs_cvp_max"                    
 [37] "vs_heart_rate_first"            "vs_heart_rate_min"             
 [39] "vs_heart_rate_max"              "vs_map_first"                  
 [41] "vs_map_min"                     "vs_map_max"                    
 [43] "vs_temp_first"                  "vs_temp_min"                   
 [45] "vs_temp_max"                    "lab_chloride_first"            
 [47] "lab_chloride_min"               "lab_chloride_max"              
 [49] "lab_chloride_abnormal"          "lab_creatinine_first"          
 [51] "lab_creatinine_min"             "lab_creatinine_max"            
 [53] "lab_creatinine_abnormal"        "lab_platelet_first"            
 [55] "lab_platelet_min"               "lab_platelet_max"              
 [57] "lab_platelet_abnormal"          "lab_potassium_first"           
 [59] "lab_potassium_min"              "lab_potassium_max"             
 [61] "lab_potassium_abnormal"         "lab_troponin_first"            
 [63] "lab_troponin_min"               "lab_troponin_max"              
 [65] "lab_troponin_abnormal"          "lab_po2_first"                 
 [67] "lab_po2_min"                    "lab_po2_max"                   
 [69] "lab_po2_abnormal"               "lab_lactate_first"             
 [71] "lab_lactate_min"                "lab_lactate_max"               
 [73] "lab_lactate_abnormal"           "lab_hemoglobin_first"          
 [75] "lab_hemoglobin_min"             "lab_hemoglobin_max"            
 [77] "lab_hemoglobin_abnormal"        "lab_sodium_first"              
 [79] "lab_sodium_min"                 "lab_sodium_max"                
 [81] "lab_sodium_abnormal"            "lab_creatinine_kinase_first"   
 [83] "lab_creatinine_kinase_min"      "lab_creatinine_kinase_max"     
 [85] "lab_creatinine_kinase_abnormal" "lab_ph_first"                  
 [87] "lab_ph_min"                     "lab_ph_max"                    
 [89] "lab_ph_abnormal"                "lab_bnp_first"                 
 [91] "lab_bnp_min"                    "lab_bnp_max"                   
 [93] "lab_bnp_abnormal"               "lab_bicarbonate_first"         
 [95] "lab_bicarbonate_min"            "lab_bicarbonate_max"           
 [97] "lab_bicarbonate_abnormal"       "lab_wbc_first"                 
 [99] "lab_wbc_min"                    "lab_wbc_max"                   
[101] "lab_wbc_abnormal"               "lab_bun_first"                 
[103] "lab_bun_min"                    "lab_bun_max"                   
[105] "lab_bun_abnormal"               "lab_pco2_first"                
[107] "lab_pco2_min"                   "lab_pco2_max"                  
[109] "lab_pco2_abnormal"              "sedative"                      
[111] "vs_cvp_flag"                    "vs

In [11]:
feature_names %>%
data.frame(feature = ., stringsAsFactors = FALSE) %>%
data.table::fwrite(file = file.path(data_dir, "features.csv"))